In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [2]:
# explain NTC, how can they vary, how are they determined in the first place, are they fixed and then exchanged

In [55]:
df_NTC = pd.read_csv("../../data/NTC.csv")
df_losses = pd.read_csv("../../data/Avtice-losses.csv")
df_renewable = pd.read_csv("../../data/Forecast-renewable-generation.csv")
df_temp = pd.read_csv("../../data/Forecast-temperature.csv")

df_losses = df_losses.drop(df_losses.index[0])

In [57]:
print(df_NTC.columns)
print(df_losses.columns)
print(df_renewable.columns)
print(df_temp.columns)

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH'],
      dtype='object')
Index(['Unnamed: 0', 'Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'solar_fore_de [MW]', 'solar_fore_it [MW]',
       'wind_fore_de [MW]', 'wind_fore_it [MW]'],
      dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [58]:
df_losses = df_losses.rename(columns={
    'Unnamed: 0': 'datetime',
})

In [59]:

df_NTC['datetime'] = pd.to_datetime(df_NTC['datetime'])
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])
df_renewable['datetime'] = pd.to_datetime(df_renewable['datetime'])
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])

In [60]:
df_losses['Wirkverluste/Active Losses'] = pd.to_numeric(df_losses['Wirkverluste/Active Losses'], errors='coerce')

In [61]:
# Convert "Zeitstempel" column to datetime format, if not already
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])

# Subtract 15 minutes from each date in the "Zeitstempel" column
df_losses['datetime'] = df_losses['datetime'] - pd.Timedelta(minutes=15)

# Print the modified dataframe
print(df_losses.head(n=10))

              datetime  Wirkverluste/Active Losses
1  2019-01-01 00:00:00                39143.346123
2  2019-01-01 00:15:00                32788.068958
3  2019-01-01 00:30:00                33018.915678
4  2019-01-01 00:45:00                34574.672936
5  2019-01-01 01:00:00                33417.095550
6  2019-01-01 01:15:00                34716.826297
7  2019-01-01 01:30:00                31063.597739
8  2019-01-01 01:45:00                30518.516417
9  2019-01-01 02:00:00                33344.466641
10 2019-01-01 02:15:00                33763.341624


In [62]:

df_losses_new = df_losses.set_index('datetime')

hourly_loss = df_losses_new.resample('1H', closed='left', label='left').sum()

In [63]:
hourly_loss

,Wirkverluste/Active Losses
datetime,
2019-01-01 00:00:00,139525.003695
2019-01-01 01:00:00,129716.036003
2019-01-01 02:00:00,133398.074458
2019-01-01 03:00:00,135133.851731
2019-01-01 04:00:00,131699.424059
...,...
2021-12-31 19:00:00,171707.317615
2021-12-31 20:00:00,159462.903412
2021-12-31 21:00:00,155109.519730
